<a href="https://colab.research.google.com/github/Nidarshan619/Planets/blob/master/Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install tslearn

     |████████████████████████████████| 793 kB 5.0 MB/s 


In [4]:
import pandas as pd
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.linear_model import LogisticRegression
from tslearn.utils import to_time_series
from tslearn.utils import to_time_series_dataset
from tslearn.clustering import TimeSeriesKMeans
from tslearn.barycenters import dtw_barycenter_averaging
from tslearn.metrics import dtw, dtw_path
from tslearn.clustering import silhouette_score

/usr/local/lib/python3.7/dist-packages/tslearn/clustering/kmeans.py:17: UserWarning: Scikit-learn <0.24 will be deprecated in a future release of tslearn
  "Scikit-learn <0.24 will be deprecated in a "


In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/Shreya_PMU/Shreya_R1.csv')

In [8]:
#df = pd.read_csv("Shreya_R1.csv")
df.head()

,Va_mag,Vb_mag,Vc_mag,Ia_mag,Ib_mag,Ic_mag,V1_mag,I1_mag,V2_mag,V0_mag,I0_mag,I2_mag,Va_ang,Vb_ang,Vc_ang,Ia_ang,Ib_ang,Ic_ang,V1_ang,I1_ang,V2_ang,V0_ang,I0_ang,I2_ang,Freq,ROCOF,Events
0,10646.684570,10646.748047,10646.700195,491.170166,491.169495,491.176086,10646.710938,491.171936,0.038324,0.000560,0.000224,0.004219,-121.217033,118.782875,-1.217377,49.385487,-70.613655,169.385956,-121.217186,49.385933,12.307068,-72.562317,47.214073,-66.098442,59.998001,0.0,0
1,10646.711914,10646.721680,10646.696289,491.171570,491.168671,491.175568,10646.709961,491.171936,0.015268,0.000735,0.000235,0.004016,-121.227867,118.771988,-1.227984,49.374691,-70.624550,169.375351,-121.227936,49.375156,-38.580269,-63.057339,43.299839,-47.018227,59.998001,0.0,0
2,10646.691406,10646.720703,10646.729492,491.170837,491.168182,491.177185,10646.713867,491.172058,0.023471,0.000662,0.000224,0.005358,-121.238754,118.761391,-1.238818,49.363773,-70.635170,169.364548,-121.238724,49.364391,70.967033,-64.459114,43.753063,-54.615639,59.998001,0.0,0
3,10646.684570,10646.747070,10646.700195,491.170135,491.169464,491.176025,10646.710938,491.171906,0.037998,0.000697,0.000234,0.004217,-121.249352,118.750557,-1.249694,49.353172,-70.645973,169.353638,-121.249489,49.353615,12.857522,-71.361115,45.862991,-66.316330,59.998001,0.0,0
4,10646.709961,10646.719727,10646.694336,491.171478,491.168579,491.175507,10646.708008,491.171844,0.014520,0.000588,0.000219,0.004026,-121.260185,118.739685,-1.260298,49.342381,-70.656876,169.343018,-121.260269,49.342842,-38.431492,-51.785282,42.666668,-47.099350,59.998001,0.0,0


In [ ]:
df.info()

In [ ]:
df["I1_ang"] = df.I1_ang-df.V1_ang
df["V2_ang"] = df.V2_ang-df.V1_ang
df["V0_ang"] = df.V0_ang-df.V1_ang
df["I0_ang"] = df.I0_ang-df.V1_ang
df["I2_ang"] = df.I2_ang-df.V1_ang
df["V_m"] = (df["Va_mag"]+df["Vb_mag"]+df["Vc_mag"])/3
df = df.drop(["V1_ang","Va_ang","Vb_ang","Vc_ang","Ia_ang","Ib_ang","Ic_ang"],axis=1)
df.shape

In [11]:
#To generate a book-keeping scheme. 
def diffe(df):
    df["diff"] = df.Events.diff()
    df.loc[pd.isnull(df["diff"]),"diff"] = 0
    df["Indexing"] = np.array(range(1,df.shape[0]+1))
    indices = list(df.loc[df["diff"]!=0,"Indexing"])
    indices.insert(0,1)
    indices.insert(len(indices),df.shape[0])
    return indices,df

In [19]:
df = df.loc[df.Events!=0]
#df_lr = df
df_lr = df.loc[ (df.Events == 8) |(df.Events == 1)]
indices,df_lr = diffe(df_lr)  #Function call
df_va = df_lr[['Va_mag','Events','diff','Indexing']]
df_va.head(184)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

,Va_mag,Events,diff,Indexing
5260,10935.034180,1,0.0,1
5261,10935.026367,1,0.0,2
5262,10934.995117,1,0.0,3
5263,10935.023438,1,0.0,4
5264,10935.016602,1,0.0,5
...,...,...,...,...
14143,10770.791992,8,0.0,180
14144,10771.239258,8,0.0,181
14145,10771.368164,8,0.0,182
14146,10771.256836,8,0.0,183


In [12]:
lower = [0,0,0,0,0,0,0,0,0,0]
upper = [88,88,88,88,88,88,88,88,88,88]
def transposed(df, df_va,indices):
    data_df = pd.DataFrame()
    temp_event = []
    for i in range(1,len(indices)):
        temp = df.loc[(df.Indexing>=indices[i-1])&(df.Indexing<indices[i])]
        t_events = temp.Events.unique()
        temp = temp.drop(["Events","diff","Indexing"],1).iloc[lower[int(temp.Events.unique())]:upper[int(temp.Events.unique())],:]
        list_temp = []
        temp_event.extend(t_events)
        for col in temp.columns:
            list_temp.extend(list(temp[col]))
        data_df[str(i)] = list_temp
    data_df = pd.DataFrame(np.transpose(np.array(data_df)))
    data_df["Event_Label"] = np.array(temp_event)
    return data_df

In [ ]:
indices

[1,
 92,
 182,
 273,
 363,
 453,
 812,
 902,
 992,
 1112,
 1202,
 1353,
 1443,
 1593,
 1683,
 1924,
 2164,
 2284,
 2374,
 2465,
 2555,
 2645,
 2795,
 2915,
 3036,
 3398,
 3518,
 3609,
 3699,
 3790,
 3880,
 3970,
 4061,
 4241,
 4331,
 4571,
 4661,
 4781,
 4902,
 4993,
 5082,
 5202,
 5292,
 5412,
 5892,
 6012,
 6102,
 6222,
 6312,
 6402,
 6492,
 6582,
 6702,
 6943,
 7033,
 7123,
 7214,
 7335,
 7516,
 7606,
 7696,
 7786,
 7876,
 7996,
 8116,
 8206,
 8296,
 8416,
 8506,
 8596,
 8686,
 8776,
 8866,
 8956,
 9047,
 9287,
 9467,
 9587,
 9767,
 9917,
 10037,
 10157,
 10247,
 10337,
 10427,
 10548,
 10638,
 10758,
 10848,
 10938,
 11028,
 11119,
 11239,
 11329,
 11480,
 11600,
 11690,
 11780,
 11870,
 11960,
 12110,
 12230,
 12470,
 12560,
 12651,
 12741,
 12981,
 13101,
 13190,
 13310,
 13431,
 13522,
 13612,
 13702,
 13793,
 13883,
 13974,
 14064,
 14305,
 14485,
 14575,
 14665,
 14786,
 14876,
 14967,
 15087,
 15208,
 15298,
 15418,
 15508,
 15628,
 15809,
 15899,
 16019,
 16109,
 16199,
 162

In [ ]:
data_df = transposed(df_lr, df_va,indices) # Function called
data_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1721,1722,1723,1724,1725,1726,1727,1728,1729,1730,1731,1732,1733,1734,1735,1736,1737,1738,1739,1740,1741,1742,1743,1744,1745,1746,1747,1748,1749,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,Event_Label
0,10696.883789,10696.912109,10696.901367,10696.882812,10696.913086,10696.903320,10696.883789,10696.914062,10696.904297,10696.885742,10696.914062,10696.905273,10696.887695,10696.916992,10696.907227,10696.888672,10696.916992,10696.908203,10696.891602,10696.917969,10696.909180,10696.891602,10696.919922,10696.910156,10696.891602,10696.920898,10696.912109,10696.892578,10696.921875,10696.912109,10696.893555,10636.926758,10425.409180,10302.900391,10216.171875,10193.641602,10336.849609,10403.583984,10441.505859,10459.955078,...,10625.564128,10638.949544,10650.360026,10659.763672,10667.180339,10672.638672,10676.225260,10678.009440,10678.084961,10676.571289,10673.602539,10669.331055,10663.976888,10657.775391,10650.987630,10643.908854,10636.853516,10630.108073,10623.975911,10618.705404,10614.498047,10611.527995,10609.896159,10609.631836,10610.711914,10613.070312,10616.560872,10621.033529,10626.304362,10632.147135,10638.380859,10644.790039,10651.177734,10657.383789,10663.270182,10668.700195,10673.586263,10677.858398,10681.465495,9
1,10782.387695,10782.404297,10782.420898,10782.380859,10782.396484,10782.413086,10782.374023,10782.391602,10782.408203,10782.372070,10782.389648,10782.407227,10782.368164,10782.386719,10782.403320,10782.367188,10782.382812,10782.399414,10782.362305,10782.378906,10782.395508,10782.357422,10782.374023,10782.392578,10782.352539,10782.372070,10782.387695,10782.350586,10782.366211,10782.380859,10782.332031,10767.974609,10751.813477,10750.512695,10749.937500,10749.535156,10749.157227,10748.880859,10748.622070,10748.329102,...,10746.432617,10746.304688,10746.178060,10746.062500,10745.961589,10745.852214,10745.762695,10745.676107,10745.584635,10745.510091,10745.434570,10745.361003,10745.289714,10745.223307,10745.154297,10745.096680,10745.046224,10744.979818,10744.930013,10744.884766,10744.826172,10744.780273,10744.742839,10744.707682,10744.672526,10744.643555,10744.615885,10744.595703,10744.580404,10744.561849,10744.550456,10744.554036,10744.546224,10744.547526,10744.557292,10744.561523,10744.569661,10744.593424,10744.604818,5
2,10749.780273,10749.785156,10749.813477,10749.778320,10749.784180,10749.813477,10749.779297,10749.784180,10749.813477,10749.779297,10749.786133,10749.813477,10749.779297,10749.785156,10749.813477,10749.779297,10749.785156,10749.813477,10749.780273,10749.786133,10749.813477,10749.780273,10749.786133,10749.813477,10749.779297,10749.786133,10749.813477,10749.780273,10749.787109,10749.814453,10749.778320,10723.731445,10600.118164,10414.449219,10324.355469,10276.395508,10405.337891,10462.187500,10501.366211,10519.865234,...,10680.097982,10693.249023,10704.485677,10713.784505,10721.138997,10726.580404,10730.198893,10732.051758,10732.226237,10730.844727,10728.039062,10723.954753,10718.801758,10712.808268,10706.229818,10699.361654,10692.491211,10685.897461,10679.894206,10674.710938,10670.544596,10667.579102,10665.903320,10665.544922,10666.500977,10668.699870,10671.998372,10676.264323,10681.308919,10686.923177,10692.916667,10699.102214,10705.275716,10711.281901,10716.973958,10722.239583,10726.985352,10731.141276,10734.644206,9
3,10737.002930,10737.041992,10737.026367,10737.005859,10737.041992,10737.027344,10737.008789,10737.042969,10737.028320,10737.009766,10737.044922,10737.028320,10737.010742,10737.045898,10737.032227,10737.013672,10737.051758,10737.034180,10737.013672,10737.046875,10737.033203,10737.012695,10737.047852,10737.035156,10737.018555,10737.055664,10737.039062,10737.021484,10737.059570,10737.044922,10735.534180,10823.212891,10904.448242,10910.153320,10914.009766,10917.232422,10920.172852,10922.979492,10925.567383,10927.984375,...,10940.910156,10941.242513,10941.498698,10941.70